# Read data from Opensearch

In [ ]:
%load_ext dotenv
%dotenv ../.env

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
from dotenv import load_dotenv

load_dotenv()
password = os.environ.get("OPENSEARCH_INITIAL_ADMIN_PASSWORD")

client = OpenSearch(
    hosts=[{"host": "localhost", "port": "9200"}],
    http_auth=("admin", password),
    use_ssl=False,  # to run locally, changed from True to False
    connection_class=RequestsHttpConnection,
    retry_on_timeout=True,
)

query = {
    "size": 6,  # Update this number X as required. It should produce the last X created entries
    "track_total_hits": True,
    "query": {"match_all": {}},
    "sort": [{"metadata.created_datetime": {"order": "desc"}}],
}

# query 1000 records from database

response = client.search(index="redbox-data-integration-chunk-current", body=query)

In [ ]:
for hit in response["hits"]["hits"]:
    print(f"Document ID: {hit['_id']}")

    metadata = hit["_source"].get("metadata")

    if "created_datetime" in metadata:
        print(f"Created on: {metadata['created_datetime']}")
    if "name" in metadata:
        print(f"Name: {metadata['name']}")
    # if 'description' in metadata:
    #     print(f"Description: {metadata['description']}")
    if "chunk_resolution" in metadata:
        print(f"Chunk Resolution: {metadata['chunk_resolution']}")
    print(f"Content [First 200 characters]:\n{hit['_source']['text'][:200]}")  # Print the first 200 characters
    print("---")

## Read index and alias names from Opensearch

In [ ]:
# Read index and alias names from database
client.indices.get_alias(name="*")

# Delete Index

In [ ]:
# client.indices.delete("redbox-data-integration-chunk") #index